<a href="https://colab.research.google.com/github/otnemrasordep/DadaGP-track-processing/blob/main/portugal_anki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Portugal Municipalities Anki Deck

What the script does:
1. Get a list of both municipalities and disctricts from Portugal;
2. Get a list of images for the location of the municipalities/districts in Portugal;
3. Download 308 images for municipalities, and 18 for districts;
4. Overlay images of municipalities with corresponding district (the latter with higher transparency);
5. Create a .csv file as an anki deck with:

    5.1 Front: overlaid map
    
    5.2 Back : municipality, DISTRICT    

In [2]:
# Import the library to mount Google Drive
from google.colab import drive

# Mount the Google Drive at /content/drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/anki_portugal-geography
!pwd

/content/drive/MyDrive/anki_portugal-geography
/content/drive/MyDrive/anki_portugal-geography


In [ ]:
from lxml import etree

def overlay_svg_files(background_svg_path, foreground_svg_path, output_svg_path, opacity=0.5):
    """
    Overlays two SVG files, adding the second as a transparent layer on top of the first.

    Parameters:
        background_svg_path (str): Path to the background SVG file.
        foreground_svg_path (str): Path to the foreground SVG file.
        output_svg_path (str): Path to save the resulting SVG.
        opacity (float): Transparency of the foreground layer (0.0 to 1.0).
    """
    # Parse the background and foreground SVG files
    background_tree = etree.parse(background_svg_path)
    foreground_tree = etree.parse(foreground_svg_path)

    # Get the root elements of both SVGs
    background_root = background_tree.getroot()
    foreground_root = foreground_tree.getroot()

    # Add an opacity attribute to the foreground SVG
    foreground_root.attrib["opacity"] = str(opacity)

    # Append the foreground to the background
    background_root.append(foreground_root)

    # Save the resulting SVG
    background_tree.write(output_svg_path, pretty_print=True, xml_declaration=True, encoding="UTF-8")
    print(f"Overlay SVG saved to {output_svg_path}")

# Example usage
overlay_svg_files(image1_path, image2_path, "result_overlay.svg", opacity=0.5)


Overlay SVG saved to result_overlay.svg


In [ ]:
# Scraping images for concelhos

In [ ]:
import pandas as pd

# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_municipalities_of_Portugal"

# Read the HTML tables from the page
tables = pd.read_html(url)

# Identify the relevant table (inspect the output to find the correct one)
for i, table in enumerate(tables):
    print(f"Table {i} preview:")
    print(table.head(), "\n")

# Assuming the target table is at index 0
municipality_table = tables[3]

# Select only the "Name" and "District/Aut. Region" columns
# Adjust column names to match the extracted table
municipality_table = municipality_table[["Name of municipality[a]", "District / Autonomous region"]]

# Rename columns for clarity
municipality_table.columns = ["Name", "District"]

# Display the cleaned-up table
print(municipality_table.head())

# Save the cleaned table to a CSV file (optional)
municipality_table.to_csv("portugal_municipalities_cleaned.csv", index=False)

print("Cleaned municipality table saved successfully!")


In [ ]:
import pandas as pd

# Path to your CSV file
file_path = '/content/drive/MyDrive/anki_portugal-geography/portugal_municipalities_cleaned.csv'

# Read the CSV file into a DataFrame
municipality_table = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(municipality_table.head())


                 Name District
0              Águeda   Aveiro
1  Albergaria-a-Velha   Aveiro
2              Anadia   Aveiro
3              Arouca   Aveiro
4              Aveiro   Aveiro


In [ ]:
# Table saved with [Concelho / Distrito]
municipality_table

,Name,District
0,Águeda,Aveiro
1,Albergaria-a-Velha,Aveiro
2,Anadia,Aveiro
3,Arouca,Aveiro
4,Aveiro,Aveiro
...,...,...
303,Porto Santo,Madeira
304,Ribeira Brava,Madeira
305,Santa Cruz,Madeira
306,Santana,Madeira


In [ ]:
# Trying to get the images

# base_link = 'https://en.wikipedia.org/wiki/'

import requests
from bs4 import BeautifulSoup

# Function to fetch image filenames from a given URL
def fetch_image_filenames(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")

        # Find all image tags
        img_tags = soup.find_all("img")

        # Extract filenames from the 'src' attributes of the image tags
        filenames = [img["src"].split("/")[-1] for img in img_tags if "src" in img.attrs]

        return filenames
    except Exception as e:
        print(f"Error fetching images from {url}: {e}")
        return []

# URL of the Wikipedia page for "Águeda"
url = "https://en.wikipedia.org/wiki/Águeda"

# Fetch and display the image filenames
image_filenames = fetch_image_filenames(url)
print("Image filenames:")
print(image_filenames)


Image filenames:
['wikipedia.png', 'wikipedia-wordmark-en.svg', 'wikipedia-tagline-en.svg', '213px-Rua_Luis_de_Cam%C3%B5es%2C_%C3%81gueda.jpg', '100px-Flag_of_%C3%81gueda.svg.png', '85px-AGD1.png', '218px-LocalAgueda.svg.png', '23px-Flag_of_Portugal.svg.png', '220px-Barcos_de_pesca_na_Pateira_de_Fermentelos-%C3%81gueda-Portugal.jpg', '22px-Flag_of_Brazil.svg.png', '23px-Flag_of_Guinea-Bissau.svg.png', '23px-Flag_of_Spain.svg.png', '23px-Flag_of_Belgium_%28civil%29.svg.png', '23px-Flag_of_Portugal.svg.png', '10px-OOjs_UI_icon_edit-ltr-progressive.svg.png', 'start?type=1x1', 'wikimedia-button.svg', 'poweredby_mediawiki.svg']


In [ ]:
pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14346 sha256=50087084ba173b4bf3b96c8a44223b37c93f74aaca56bfab6cb41ba98140c13a
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


In [ ]:
pip install cairosvg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00


In [1]:
import requests
from bs4 import BeautifulSoup

# Function to fetch image URLs using requests and custom user-agent
def get_image_urls_using_requests(wikipedia_page):
    base_url = f"https://en.wikipedia.org/wiki/{wikipedia_page.replace(' ', '_')}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    # Send GET request
    response = requests.get(base_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract image URLs
    image_urls = []
    for img_tag in soup.find_all("img"):
        img_url = img_tag.get("src")
        if img_url:
            full_url = "https:" + img_url
            image_urls.append(full_url)

    return image_urls

# Example for the "Águeda" page
municipality_page = "Aveiro, Portugal"
image_urls = get_image_urls_using_requests(municipality_page)

# Print all the image URLs
for url in image_urls:
    print(url)


https:/static/images/icons/wikipedia.png
https:/static/images/mobile/copyright/wikipedia-wordmark-en.svg
https:/static/images/mobile/copyright/wikipedia-tagline-en.svg
https://upload.wikimedia.org/wikipedia/commons/thumb/a/a1/Ilha_Dos_Puxadoiros_%2847261194681%29_%28cropped%29.jpg/280px-Ilha_Dos_Puxadoiros_%2847261194681%29_%28cropped%29.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/c/cf/Aveiro_Foto_%28cropped%29.jpg/139px-Aveiro_Foto_%28cropped%29.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/d/dc/Passeio_de_moliceiro_%28cropped%29.jpg/139px-Passeio_de_moliceiro_%28cropped%29.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/9/94/Antiga_Capitania_do_Porto_de_Aveiro_-_Portugal_%285679861484%29_%28cropped%29.jpg/139px-Antiga_Capitania_do_Porto_de_Aveiro_-_Portugal_%285679861484%29_%28cropped%29.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/0/0a/Conjunto_de_Armaz%C3%A9ns_do_Sal_do_Canal_de_S%C3%A3o_Roque_ou_Palheiros_de_sal_do_Canal_de_S%C3%A3o_Roq

In [ ]:
# Filter and save URLs that contain "Local"
filtered_urls = [url for url in image_urls if "Local" in url]

# Print the filtered URLs
for url in filtered_urls:
    print(url)

https://upload.wikimedia.org/wikipedia/commons/thumb/3/3b/LocalIlhavo.svg/218px-LocalIlhavo.svg.png


In [ ]:
# To remove accents from words
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.2 MB/s eta 0:00:00


In [ ]:
#

import unicodedata

def remove_accents(input_str):
    # Decompose characters into base and combining marks
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    cleaned_str = ''.join(
        c for c in nfkd_form
        if not unicodedata.combining(c) or c.lower() == 'ç'
    )
    return cleaned_str

# Test cases
word = "Águeda"
clean_word = remove_accents(word)
print(clean_word)  # Output: Agueda

word_with_cedilla = "aço"
clean_word_with_cedilla = remove_accents(word_with_cedilla)
print(clean_word_with_cedilla)  # Output: ação

word_with_upper_cedilla = "Ação"
clean_word_with_upper_cedilla = remove_accents(word_with_upper_cedilla)
print(clean_word_with_upper_cedilla)  # Output: Ação


Agueda
aco
Acao


In [ ]:
import time
from unidecode import unidecode

# Add a new column to store the image links, initialize it with empty lists
municipality_table["Image Link"] = None

for index, row in municipality_table.iterrows():
    # Normalize the municipality name to remove accents
    municipality_name = remove_accents(row["Name"])
    print(f"Processing Municipality: {municipality_name}")

    # Construct alternative names
    alt1_mun_name = municipality_name + ' (Portugal)'
    alt2_mun_name = municipality_name + ', Portugal'

    # Get image URLs for the primary name
    image_urls = get_image_urls_using_requests(municipality_name)
    filtered_urls = [url for url in image_urls if "Local" in url]

    # Add a short delay after the first request
    time.sleep(1)

    # Try alternative names if no URLs are found
    if not filtered_urls:
        image_urls = get_image_urls_using_requests(alt1_mun_name)
        filtered_urls = [url for url in image_urls if "Local" in url]
        time.sleep(1)  # Add delay after the second request

    if not filtered_urls:
        image_urls = get_image_urls_using_requests(alt2_mun_name)
        filtered_urls = [url for url in image_urls if "Local" in url]
        time.sleep(1)  # Add delay after the third request

    # Store the filtered URLs in the "Image Link" column
    municipality_table.at[index, "Image Link"] = filtered_urls

# Display the updated DataFrame
print(municipality_table)


Processing Municipality: Agueda
Processing Municipality: Albergaria-a-Velha
Processing Municipality: Anadia
Processing Municipality: Arouca
Processing Municipality: Aveiro
Processing Municipality: Castelo de Paiva
Processing Municipality: Espinho
Processing Municipality: Estarreja
Processing Municipality: Ilhavo
Processing Municipality: Mealhada
Processing Municipality: Murtosa
Processing Municipality: Oliveira de Azemeis
Processing Municipality: Oliveira do Bairro
Processing Municipality: Ovar
Processing Municipality: Santa Maria da Feira
Processing Municipality: Sao Joao da Madeira
Processing Municipality: Sever do Vouga
Processing Municipality: Vagos
Processing Municipality: Vale de Cambra
Processing Municipality: Aljustrel
Processing Municipality: Almodovar
Processing Municipality: Alvito
Processing Municipality: Barrancos
Processing Municipality: Beja
Processing Municipality: Castro Verde
Processing Municipality: Cuba
Processing Municipality: Ferreira do Alentejo
Processing Munici

In [ ]:
municipality_table.to_csv("portugal_municipalities_wlinks.csv", index=False)

In [ ]:
# AFTER FIXING MISSING VALUES MANUALLY
# TURNED OUT THE SCRIPT WAS ONLY WORKING FOR PORTUGAL CONTINETAL

import pandas as pd

# Path to your CSV file
file_path = '/content/drive/MyDrive/anki_portugal-geography/portugal_municipalities_wlinks_clean - portugal_municipalities_wlinks_clean.csv'

# Read the CSV file into a DataFrame
municipality_table_final = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(municipality_table_final.head())

                 Name District  \
0              Águeda   Aveiro   
1  Albergaria-a-Velha   Aveiro   
2              Anadia   Aveiro   
3              Arouca   Aveiro   
4              Aveiro   Aveiro   

                                          Image Link  \
0  ['https://upload.wikimedia.org/wikipedia/commo...   
1  ['https://upload.wikimedia.org/wikipedia/commo...   
2  ['https://upload.wikimedia.org/wikipedia/commo...   
3  ['https://upload.wikimedia.org/wikipedia/commo...   
4  ['https://upload.wikimedia.org/wikipedia/commo...   

                                    Image Link Clean  
0  https://upload.wikimedia.org/wikipedia/commons...  
1  https://upload.wikimedia.org/wikipedia/commons...  
2  https://upload.wikimedia.org/wikipedia/commons...  
3  https://upload.wikimedia.org/wikipedia/commons...  
4  https://upload.wikimedia.org/wikipedia/commons...  


In [ ]:


link = municipality_table_final['Image Link'][0]
link = link.strip("[]").strip("'")
print(link)
link = link.split('/thumb')[0] + link.split('/thumb')[1]
print(link)
link = link.rsplit('/', 1)[0]
print(link)

https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/LocalAgueda.svg/218px-LocalAgueda.svg.png
https://upload.wikimedia.org/wikipedia/commons/c/c6/LocalAgueda.svg/218px-LocalAgueda.svg.png
https://upload.wikimedia.org/wikipedia/commons/c/c6/LocalAgueda.svg


In [ ]:
# Get final links for high resolution image
# Function to clean the Image Link
def clean_image_link(image_link):
    # Ensure the input is a string and strip the list-like characters
    link = image_link.strip("[]").strip("'")

    # Remove '/thumb' and everything after '.svg'
    if '/thumb' in link:
        link = link.split('/thumb')[0] + link.split('/thumb')[1]

    # Remove everything after the last '/'
    link = link.rsplit('/', 1)[0]

    return link

# Apply the function to the 'Image Link' column and create 'Image Link Clean'
municipality_table_final['Image Link Clean'] = municipality_table_final['Image Link'].apply(clean_image_link)

# Display the updated DataFrame
print(municipality_table_final['Image Link Clean'][100])



https://upload.wikimedia.org/wikipedia/commons/6/64/LocalVilaVicosa.svg


In [ ]:
municipality_table_final.to_csv("portugal_municipalities_wlinks_clean.csv", index=False)

In [7]:
# AFTER FIXING MISSING VALUES MANUALLY
# TURNED OUT THE SCRIPT WAS ONLY WORKING FOR PORTUGAL CONTINETAL

import pandas as pd

# Path to your CSV file
file_path = '/content/drive/MyDrive/anki_portugal-geography/portugal_municipalities_wlinks_clean_FINAL.csv'

# Read the CSV file into a DataFrame
municipality_table_final = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(municipality_table_final.head())

                 Name District  \
0              Águeda   Aveiro   
1  Albergaria-a-Velha   Aveiro   
2              Anadia   Aveiro   
3              Arouca   Aveiro   
4              Aveiro   Aveiro   

                                          Image Link  \
0  ['https://upload.wikimedia.org/wikipedia/commo...   
1  ['https://upload.wikimedia.org/wikipedia/commo...   
2  ['https://upload.wikimedia.org/wikipedia/commo...   
3  ['https://upload.wikimedia.org/wikipedia/commo...   
4  ['https://upload.wikimedia.org/wikipedia/commo...   

                                    Image Link Clean  
0  https://upload.wikimedia.org/wikipedia/commons...  
1  https://upload.wikimedia.org/wikipedia/commons...  
2  https://upload.wikimedia.org/wikipedia/commons...  
3  https://upload.wikimedia.org/wikipedia/commons...  
4  https://upload.wikimedia.org/wikipedia/commons...  


Images have very poor resolution - trying a different approach.

In [ ]:
!pip install requests beautifulsoup4


In [ ]:
pip install cairosvg

In [ ]:
import os
import requests

# Directory to save downloaded images
output_dir = "/content/drive/MyDrive/anki_portugal-geography/municipality_maps/source-municipalities"
os.makedirs(output_dir, exist_ok=True)

# Custom headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Iterate over rows in the DataFrame
for index, row in municipality_table_final.iterrows():
    # Get the cleaned link from the 'Image Link Clean' column
    link = row['Image Link Clean']

    # Construct the download URL by appending "?download"
    svg_url = f"{link}?download"

    # Extract the filename from the URL
    filename = os.path.basename(link)

    # Download the file
    try:
        response = requests.get(svg_url, headers=headers, stream=True)
        response.raise_for_status()

        # Save the file
        file_path = os.path.join(output_dir, filename)
        with open(file_path, "wb") as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
        print(f"Downloaded: {filename}")
    except Exception as e:
        print(f"Failed to download {filename}: {e}")


Downloaded: LocalAgueda.svg
Downloaded: LocalAlbergariaAVelha.svg
Downloaded: LocalAnadia.svg
Downloaded: LocalArouca.svg
Downloaded: LocalAveiro.svg
Downloaded: LocalCasteloDePaiva.svg
Downloaded: LocalEspinho.svg
Downloaded: LocalEstarreja.svg
Downloaded: LocalIlhavo.svg
Downloaded: LocalMealhada.svg
Downloaded: LocalMurtosa.svg
Downloaded: LocalOliveiraDeAzemeis.svg
Downloaded: LocalOliveiraDoBairro.svg
Downloaded: LocalOvar.svg
Downloaded: LocalSantaMariaDaFeira.svg
Downloaded: LocalSaoJoaoDaMadeira.svg
Downloaded: LocalSeverDoVouga.svg
Downloaded: LocalVagos.svg
Downloaded: LocalValeDeCambra.svg
Downloaded: LocalAljustrel.svg
Downloaded: LocalAlmodovar.svg
Downloaded: LocalAlvito.svg
Downloaded: LocalBarrancos.svg
Downloaded: LocalBeja.svg
Downloaded: LocalCastroVerde.svg
Downloaded: LocalCuba.svg
Downloaded: LocalFerreiraDoAlentejo.svg
Downloaded: LocalMertola.svg
Downloaded: LocalMoura.svg
Downloaded: LocalOdemira.svg
Downloaded: LocalOurique.svg
Downloaded: LocalSerpa.svg
Downl

In [ ]:
len(os.listdir('/content/drive/MyDrive/anki_portugal-geography/municipality_maps/source-municipalities'))

279

In [ ]:
# THE LINKS I'M LOOKING FOR ARE ONE DIRECTORY BEFORE THE LINKS ON THE MUNICIPALITY_TABLE (AND REMOVE 'THUMB' )

In [ ]:
# I JUST NEED TO "PRESS" THE DOWNLOAD BUTTON FOR THOSE

In [6]:
from lxml import etree

def overlay_svg_files(background_svg_path, foreground_svg_path, output_svg_path, opacity=0.5):
    """
    Overlays two SVG files, adding the second as a transparent layer on top of the first.

    Parameters:
        background_svg_path (str): Path to the background SVG file.
        foreground_svg_path (str): Path to the foreground SVG file.
        output_svg_path (str): Path to save the resulting SVG.
        opacity (float): Transparency of the foreground layer (0.0 to 1.0).
    """
    # Parse the background and foreground SVG files
    background_tree = etree.parse(background_svg_path)
    foreground_tree = etree.parse(foreground_svg_path)

    # Get the root elements of both SVGs
    background_root = background_tree.getroot()
    foreground_root = foreground_tree.getroot()

    # Add an opacity attribute to the foreground SVG
    foreground_root.attrib["opacity"] = str(opacity)

    # Append the foreground to the background
    background_root.append(foreground_root)

    # Save the resulting SVG
    background_tree.write(output_svg_path, pretty_print=True, xml_declaration=True, encoding="UTF-8")
    print(f"Overlay SVG saved to {output_svg_path}")


In [ ]:
image2_path = '/content/drive/MyDrive/anki_portugal-geography/municipality_maps/source-districts/LocalDistritoViseu.svg'
image1_path = '/content/drive/MyDrive/anki_portugal-geography/municipality_maps/source-municipalities/LocalLamego.svg'
output_path = '/content/drive/MyDrive/anki_portugal-geography/municipality_maps/overlaid/LocalLamego.svg'

In [ ]:
# Example usage
overlay_svg_files(image1_path, image2_path, output_path, opacity=0.5)


Overlay SVG saved to /content/drive/MyDrive/anki_portugal-geography/municipality_maps/overlaid/LocalLamego.svg


In [16]:

district_path = '/content/drive/MyDrive/anki_portugal-geography/municipality_maps/source-districts'
mun_path = '/content/drive/MyDrive/anki_portugal-geography/municipality_maps/source-municipalities'
output_dir = '/content/drive/MyDrive/anki_portugal-geography/municipality_maps/overlaid'

for row in municipality_table_final.iterrows():
    mun_img = row[1]['Image Link Clean'].split('/')[-1]
    dis_img = 'LocalDistrito' + row[1]['District'] + '.svg'
    # print(f'Mun: {mun_img}')
    # print(f'Dis: {dis_img}')

    mun_img_path = os.path.join(mun_path, mun_img)
    # print(f'Mun: {mun_img_path}')
    dis_img_path = os.path.join(district_path, dis_img)
    # print(f'Dis: {dis_img_path}')

    output_path = os.path.join(output_dir, mun_img)
    # print(f'Out: {output_path}')

    try:
        overlay_svg_files(mun_img_path, dis_img_path, output_path, opacity=0.5)
        # print(f'Saved to {output_path}')
    except:
        print(f'Error with {row[1]["Name"]}')



Overlay SVG saved to /content/drive/MyDrive/anki_portugal-geography/municipality_maps/overlaid/LocalAgueda.svg
Overlay SVG saved to /content/drive/MyDrive/anki_portugal-geography/municipality_maps/overlaid/LocalAlbergariaAVelha.svg
Overlay SVG saved to /content/drive/MyDrive/anki_portugal-geography/municipality_maps/overlaid/LocalAnadia.svg
Overlay SVG saved to /content/drive/MyDrive/anki_portugal-geography/municipality_maps/overlaid/LocalArouca.svg
Overlay SVG saved to /content/drive/MyDrive/anki_portugal-geography/municipality_maps/overlaid/LocalAveiro.svg
Overlay SVG saved to /content/drive/MyDrive/anki_portugal-geography/municipality_maps/overlaid/LocalCasteloDePaiva.svg
Overlay SVG saved to /content/drive/MyDrive/anki_portugal-geography/municipality_maps/overlaid/LocalEspinho.svg
Overlay SVG saved to /content/drive/MyDrive/anki_portugal-geography/municipality_maps/overlaid/LocalEstarreja.svg
Overlay SVG saved to /content/drive/MyDrive/anki_portugal-geography/municipality_maps/over

In [18]:
import os

len(os.listdir('/content/drive/MyDrive/anki_portugal-geography/municipality_maps/overlaid'))

278

In [17]:
# adding final image 'Santo Tirso'

path = '/content/drive/MyDrive/anki_portugal-geography/LocalSantoTirso.svg'
st_dist_path = '/content/drive/MyDrive/anki_portugal-geography/municipality_maps/source-districts/LocalDistritoPorto.svg'
st_out_path = '/content/drive/MyDrive/anki_portugal-geography/municipality_maps/overlaid/LocalSantoTirso.svg'

overlay_svg_files(path, st_dist_path, st_out_path, opacity=0.5)

Overlay SVG saved to /content/drive/MyDrive/anki_portugal-geography/municipality_maps/overlaid/LocalSantoTirso.svg
